In [2]:
import os
import sys


spark_path = "C:\spark-2.1.0"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext("local", "test")

In [3]:
#import os
#import sys

#os.environ['SPARK_HOME'] = "C:/spark-2.1.0"
#spark_home = os.environ.get('SPARK_HOME', None)
#sys.path.insert(0, os.path.join(spark_home, 'C:/spark-2.1.0/python'))
#sys.path.insert(0, os.path.join(spark_home, 'C:/spark-2.1.0/python/lib/py4j-0.9-src.zip'))
#print  (os.path.join(spark_home, 'C:/spark-2.1.0/python/pyspark/shell.py'))
#execfile(os.path.join(spark_home, 'C:\spark-2.1.0\python\pyspark\shell.py'))


# Moteur de recommandation de films

On va générer un moteur de recommandation dans Spark. Pour cela on va utiliser le dataset mis à disposition par movieLens.

In [4]:
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
import pandas as pd
import seaborn as sns
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS

#use a parsing function
def parseRating(line):
    fields = line.strip().split("\t")
    return long(fields[3]) % 10, (int(fields[0]), int(fields[1]), int(fields[2]))


In [5]:
workers = pd.read_csv("extract_attack_worker_demographics.tsv", delimiter="\t")
workers.head()

,worker_id,gender,english_first_language,age_group,education
0,833,female,0,45-60,bachelors
1,1072,male,0,30-45,bachelors
2,872,male,0,18-30,hs
3,2116,male,0,30-45,professional
4,453,male,0,30-45,hs


In [6]:
#create an RDD (Resilient Distributed Datasets)
                             # les données (ici ma liste de notes), numSlices    
myWorkersRDD = sc.parallelize(workers["worker_id"], 1)

In [7]:
# .collect() pour recuperer les données distribuées sur les worker nodes 
myWorkersRDD.collect()

[833,
 1072,
 872,
 2116,
 453,
 164,
 301,
 77,
 865,
 715,
 1852,
 344,
 916,
 407,
 2912,
 10,
 177,
 1704,
 272,
 1343,
 3038]

Chargement du dataset movieLens:

In [8]:
revisions = pd.read_csv("attack_annotated_comments.tsv", delimiter="\t")
revisions.head()

,rev_id,comment,year,logged_in,ns,sample,split
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train
3,89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev
4,93890,This page will need disambiguation.,2002,True,article,random,train


In [12]:
myRevisionsRDD = sc.parallelize(revisions["rev_id"], 1)
commentaires = sc.parallelize(revisions["comment"], 1)

In [10]:
# .collect() pour recuperer les données distribuées sur les worker nodes 
myRevisionsRDD.collect()

[37675,
 44816,
 49851,
 89320,
 93890,
 102817,
 103624,
 111032,
 120283,
 128532,
 133562,
 138117,
 155243,
 177310,
 192579,
 201190,
 208009,
 249432,
 252031,
 268558,
 276906,
 286174,
 290598,
 294124,
 297866,
 317177,
 336654,
 344567,
 356383,
 358984,
 360594,
 375422,
 376205,
 377054,
 386473,
 388807,
 398923,
 421948,
 460946,
 462765,
 537948,
 546902,
 554005,
 569192,
 583986,
 586452,
 593977,
 600918,
 622095,
 633526,
 661509,
 680470,
 694840,
 700512,
 707010,
 709074,
 724915,
 749616,
 783899,
 784353,
 801279,
 804607,
 808184,
 810614,
 811924,
 823763,
 833825,
 840118,
 849920,
 907749,
 916927,
 920274,
 923974,
 943685,
 945573,
 946822,
 966578,
 967899,
 968361,
 977682,
 996869,
 999587,
 1028038,
 1062754,
 1064649,
 1078251,
 1078839,
 1084843,
 1100463,
 1106855,
 1111314,
 1126594,
 1133583,
 1140231,
 1173977,
 1185345,
 1212997,
 1220372,
 1221055,
 1226695,
 1228544,
 1233393,
 1240138,
 1244940,
 1250844,
 1271243,
 1272192,
 1274893,
 128730

In [13]:
commentaires.collect()

["`-NEWLINE_TOKENThis is not ``creative``.  Those are the dictionary definitions of the terms ``insurance`` and ``ensurance`` as properly applied to ``destruction``.  If you don't understand that, fine, legitimate criticism, I'll write up ``three man cell`` and ``bounty hunter`` and then it will be easy to understand why ``ensured`` and ``insured`` are different - and why both differ from ``assured``.NEWLINE_TOKENNEWLINE_TOKENThe sentence you quote is absolutely neutral.  You just aren't familiar with the underlying theory of strike-back (e.g. submarines as employed in nuclear warfare) guiding the insurance, nor likely the three man cell structure that kept the IRA from being broken by the British.  If that's my fault, fine, I can fix that to explain.  But ther'es nothing ``personal`` or ``creative`` about it.NEWLINE_TOKENNEWLINE_TOKENI'm tired of arguing with you.  Re: the other article, ``multi-party`` turns up plenty, and there is more use of ``mutually`` than ``mutual``.  If I were

In [11]:
dataset = pd.read_csv("attack_annotations.tsv", delimiter="\t")
dataset.head()

,rev_id,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
0,37675,1362,0.0,0.0,0.0,0.0,0.0
1,37675,2408,0.0,0.0,0.0,0.0,0.0
2,37675,1493,0.0,0.0,0.0,0.0,0.0
3,37675,1439,0.0,0.0,0.0,0.0,0.0
4,37675,170,0.0,0.0,0.0,0.0,0.0


In [11]:
# movieLensHomeDir = "C:/Users/thaunayf/Documents/Formation_Python/Recommandation/ratings.dat"
# # ratings is an RDD of (last digit of timestamp, (userID, movieID, rating))
# # on envoie la fonction "parseRating" sur les worker nodes 
# # map = transformation (données distantes, lazy mode)
# ratings = sc.textFile(movieLensHomeDir).map(parseRating)
# # count = action, ramène un résultat 
# numRatings = ratings.count()
# numRatings


1000209

In [12]:
# definition d'une fonction pour parser une ligne du fichier movies.dat
def parseMovie(line):
    fields=line.strip().split("::")
    return int(fields[0]), fields[1]

# sc.textFile =  fichier -> RDD
# .map(parseMovie) -> pour parser le RDD
# .collect()  -> retourne une liste de tuples (id, nom)
# dict -> transforme la liste de tuple en un dictionnaire

movies = dict(sc.textFile("C:/Users/thaunayf/Documents/Formation_Python/Recommandation/movies.dat").map(parseMovie).collect())

In [13]:
movies

{1: u'Toy Story (1995)',
 2: u'Jumanji (1995)',
 3: u'Grumpier Old Men (1995)',
 4: u'Waiting to Exhale (1995)',
 5: u'Father of the Bride Part II (1995)',
 6: u'Heat (1995)',
 7: u'Sabrina (1995)',
 8: u'Tom and Huck (1995)',
 9: u'Sudden Death (1995)',
 10: u'GoldenEye (1995)',
 11: u'American President, The (1995)',
 12: u'Dracula: Dead and Loving It (1995)',
 13: u'Balto (1995)',
 14: u'Nixon (1995)',
 15: u'Cutthroat Island (1995)',
 16: u'Casino (1995)',
 17: u'Sense and Sensibility (1995)',
 18: u'Four Rooms (1995)',
 19: u'Ace Ventura: When Nature Calls (1995)',
 20: u'Money Train (1995)',
 21: u'Get Shorty (1995)',
 22: u'Copycat (1995)',
 23: u'Assassins (1995)',
 24: u'Powder (1995)',
 25: u'Leaving Las Vegas (1995)',
 26: u'Othello (1995)',
 27: u'Now and Then (1995)',
 28: u'Persuasion (1995)',
 29: u'City of Lost Children, The (1995)',
 30: u'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: u'Dangerous Minds (1995)',
 32: u'Twelve Monkeys (1995)',
 33: u'Wings 

## 1 - Exercice

* Combien d'utilisateurs ?
* Combien de films ?

In [14]:
# D'abord les utilisateurs

ratings.take(10)


[(0L, (1, 1193, 5)),
 (9L, (1, 661, 3)),
 (8L, (1, 914, 3)),
 (5L, (1, 3408, 4)),
 (1L, (1, 2355, 5)),
 (8L, (1, 1197, 3)),
 (9L, (1, 1287, 5)),
 (9L, (1, 2804, 5)),
 (8L, (1, 594, 4)),
 (8L, (1, 919, 4))]

In [15]:
# Le champ tuple[1][0] designe le user_id
# fonction lambda
ratings.map(lambda monTuple : monTuple[1][0]).distinct().count()


6040

In [16]:
# Meme chose pour les films
ratings.map(lambda monTuple : monTuple[1][1]).distinct().count()

3706

## 2 - Spliter les données
Séparer les données en un jeu de trainning (60%), validation (20%) et testing (20%) en utilisant le dernier chiffre du timestamp.
Etape 2 : répartir chaque jeu de données sur chacun des RDD

In [19]:
# ratings : format defini par la fonction parseRating
#  timestamp %10, (user_id, film_id, note)

# 60% du jeu de données, auquel on rajoute les donnees perso MyRatingsRDD
# .repartition(2) : le nouveau RDD est divisé en 2 partitions
# .cache : persist le nouveau RDD pour eviter que cette transformation de filter soit opérée pour chaque action
training = ratings.filter(lambda monTuple : monTuple[0]<6).values().union(myRatingsRDD).repartition(2).cache()

numTraining = training.count()


In [20]:
validation = ratings.filter(lambda monTuple : monTuple[0]>=6 and monTuple[0]<8).values().repartition(2).cache()
numValidation = validation.count()


In [21]:
test = ratings.filter(lambda monTuple : monTuple[0]>=8).values().repartition(2).cache()
numTest = test.count()


# La mesure de l'erreur de prédiction

In [22]:
# fonction de calcul du root-mean-square-error
def computeRmse(model, data, n):
    #get the prediction from an user - movie pair
    predictions = model.predictAll(data.map(lambda x: (x[0], x[1])))
    
    #compare  the predictions to the actual ratings
    #perform an inner join of the predictions with the data and their ratings (x[2])
        
    predictionsAndRatings = predictions.map(lambda x: ((x[0], x[1]), x[2])) \
        .join(data.map(lambda x: ((x[0], x[1]), x[2]))) \
        .values()  #keep the two ratings
    
    # Pour chaque jeu de prédiction - vraie notation, transformation en (x[0] - x[1])**2) (transformation map)
    # puis action reduce 
    return sqrt(predictionsAndRatings.map(lambda x: (x[0] - x[1])**2).reduce(add) / float(n))

In [23]:
#Ranges of variables for the models
ranks = [8, 12]
lambdas = [0.1, 10.0]
numIters = [10, 20]

#Storing the best parameters
bestModel = None
bestTestRmse = 1.0e30000
bestRank = 0
bestLambda = -0.1
bestNumIter = -1

In [24]:
#Collaborative filtering is commonly used for recommender systems.
#These techniques aim to fill in the missing entries of a user-item association matrix.
#spark.mllib currently supports model-based collaborative filtering,
#in which users and products are described by a small set of latent factors that can be used to predict missing entries.
#spark.mllib uses the alternating least squares (ALS) algorithm to learn these latent factors.


# Tres bonne alternative à une boucle sur chacun des 3 paramètres
#Training the ALS models on the training RDDs
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    #cartesian product, equivalent to neasted for loops
    
    #On entraine le modele avec le jeu d'entrainement
    model = ALS.train(training, rank, numIter, lmbda)
    
    # On estime l'erreur grace au jeu de test
    #estimate error
    testRmse = computeRmse(model, test, numTest)
    
    print "RMSE (test) = %f for the model trained with " % testRmse + \
          "rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter)
        
    #store the best settings
    if(testRmse < bestTestRmse):
        bestModel = model
        bestTestRmse = testRmse
        bestRank = rank
        bestLambda = lmbda
        bestNumIter = numIter
        
    # evaluate the best model on the validation set
    validationRmse = computeRmse(bestModel, validation, numValidation)
    
    print "The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
    + "and numIter = %d, and its RMSE on the validation set is %f." % (bestNumIter, bestTestRmse)
    

RMSE (test) = 0.875664 for the model trained with rank = 8, lambda = 0.1, and numIter = 10.
The best model was trained with rank = 8 and lambda = 0.1, and numIter = 10, and its RMSE on the validation set is 0.875664.
RMSE (test) = 0.870157 for the model trained with rank = 8, lambda = 0.1, and numIter = 20.
The best model was trained with rank = 8 and lambda = 0.1, and numIter = 20, and its RMSE on the validation set is 0.870157.
RMSE (test) = 3.752548 for the model trained with rank = 8, lambda = 10.0, and numIter = 10.
The best model was trained with rank = 8 and lambda = 0.1, and numIter = 20, and its RMSE on the validation set is 0.870157.
RMSE (test) = 3.752548 for the model trained with rank = 8, lambda = 10.0, and numIter = 20.
The best model was trained with rank = 8 and lambda = 0.1, and numIter = 20, and its RMSE on the validation set is 0.870157.
RMSE (test) = 0.874630 for the model trained with rank = 12, lambda = 0.1, and numIter = 10.
The best model was trained with rank 

## Utiliser le meilleur model pour faire les prédictions

In [25]:
# On ajoute les resultats du jeu de test a ceux du jeu d'entrainement
#compare the best model with a naive baseline always returning the mean rating
meanRating = training.union(test).map(lambda x: x[2]).mean()
baselineRmse = sqrt(validation.map(lambda x: (meanRating - x[2]) ** 2).reduce(add) / numValidation)
improvement = (baselineRmse - validationRmse) / baselineRmse * 100
print "The best model improves the baseline by %.2f" % (improvement) + "%."

The best model improves the baseline by 22.06%.


In [26]:
# load personal ratings
f = open("C:/Users/thaunayf/Documents/Formation_Python/Recommandation/personalRatings.txt")

#get only rated movies (i.e. fields[2] > 0)
myRatings = filter(lambda r: r[2] > 0, [parseRating(line)[1] for line in f])

f.close()
myRatings

[(0, 1, 4),
 (0, 780, 2),
 (0, 590, 3),
 (0, 1210, 5),
 (0, 648, 3),
 (0, 344, 1),
 (0, 165, 2),
 (0, 153, 4),
 (0, 597, 2),
 (0, 1580, 3),
 (0, 231, 4)]

In [27]:
#make personalized recommendations
myRatedMoviesIds = set([x[1] for x in myRatings])
candidates = sc.parallelize([m for m in movies if m not in myRatedMoviesIds])
predictions = bestModel.predictAll(candidates.map(lambda x: (0,x))).collect()
recommendations = sorted(predictions, key=lambda x: x[2], reverse=True)[:10]

print "Movies recommended for you:"
for i in xrange(len(recommendations)):
    print("%2d: %s" % (i+1, movies[recommendations[i][1]])).encode('ascii','ignore')
    

Movies recommended for you:
 1: Bewegte Mann, Der (1994)
 2: Fever Pitch (1997)
 3: Star Wars: Episode V - The Empire Strikes Back (1980)
 4: Bandits (1997)
 5: Hard Core Logo (1996)
 6: Anne Frank Remembered (1995)
 7: Sanjuro (1962)
 8: Star Wars: Episode IV - A New Hope (1977)
 9: Paths of Glory (1957)
10: Circus, The (1928)


In [28]:
sc.stop()